In [1]:
# specify the aws profile and region you want to use to deploy the stack
aws_profile="bbdev"
region_name="us-east-2"

In [10]:
lambda_name = !docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/infra/ \
    cloudfuse/cloud-reader-terraform output lambda_arn
lambda_name = lambda_name[0][1:len(lambda_name[0])-1]
print('lambda_name:', lambda_name)

lambda_name: arn:aws:lambda:us-east-2:615900053518:function:cloud-reader-benchmark


In [14]:
import boto3
import json

session = boto3.Session(profile_name=aws_profile)
client = session.client('lambda', region_name = region_name)

inputParams = {}

response = client.invoke(
    FunctionName = lambda_name,
    InvocationType = 'RequestResponse',
    Payload = json.dumps(inputParams)
)

responseFromChild = json.load(response['Payload'])

print('\n')
print(responseFromChild)



{'message': 'Hello, world!'}
